In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl
/kaggle/input/torch_geometric-2.5.3-py3-none-any.whl
/kaggle/input/torch_cluster-1.6.3-cp310-cp310-linux_x86_64.whl
/kaggle/input/torch_spline_conv-1.2.2-cp310-cp310-linux_x86_64.whl
/kaggle/input/torch_sparse-0.6.18-cp310-cp310-linux_x86_64.whl


In [2]:
!git clone https://github.com/rampasek/GraphGPS.git

Cloning into 'GraphGPS'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (316/316), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 526 (delta 254), reused 225 (delta 225), pack-reused 210 (from 1)
Receiving objects: 100% (526/526), 12.93 MiB | 32.37 MiB/s, done.
Resolving deltas: 100% (336/336), done.


In [3]:
%cd /kaggle/working/GraphGPS
!ls 

/kaggle/working/GraphGPS
GraphGPS.png  README.md  final-results.zip  main.py  setup.py  unittests
LICENSE       configs	 graphgps	    run      tests


##  installation for slurm if needed using conda 

In [4]:
# # for slurm if needed using conda 
# conda create -n graphgps python=3.10
# conda activate graphgps

# conda install pytorch=1.13 torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia
# conda install pyg=2.2 -c pyg -c conda-forge
# pip install pyg-lib -f https://data.pyg.org/whl/torch-1.13.0+cu117.html

# # RDKit is required for OGB-LSC PCQM4Mv2 and datasets derived from it.  
# conda install openbabel fsspec rdkit -c conda-forge

# pip install pytorch-lightning yacs torchmetrics
# pip install performer-pytorch
# pip install tensorboardX
# pip install ogb
# pip install wandb

# conda clean --all

# installation for kaggle

In [5]:
# Check existing environment
import sys
print("Python version:", sys.version)
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118

!pip install torch_geometric

# Optional dependencies:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cu124.html
# Install other required packages
!pip install pytorch-lightning yacs torchmetrics
!pip install performer-pytorch
!pip install tensorboardX
!pip install ogb
!pip install wandb

# Attempt to install RDKit (may have limitations)
!pip install rdkit-pypi
# !pip install openbabel fsspec
!pip install  fsspec

# Clean up pip cache (optional)
!pip cache purge


Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
PyTorch version: 2.4.0
CUDA available: True
CUDA version: 12.3
Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Successfully uninstalled torch-2.4.0
Found existing installation: torchvision 0.19.0
Uninstalling torchvision-0.19.0:
  Successfully uninstalled torchvision-0.19.0
Found existing installation: torchaudio 2.4.0
Uninstalling torchaudio-2.4.0:
  Successfully uninstalled torchaudio-2.4.0
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 449.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 80.8 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 75.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 25.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 67.9 MB/s eta 

In [6]:
import sys
print("Python version:", sys.version)
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)


Python version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:45:18) [GCC 12.3.0]
PyTorch version: 2.4.0
CUDA available: True
CUDA version: 12.3


In [7]:
import torch_geometric
print("PyG version:", torch_geometric.__version__)

import pytorch_lightning
print("PyTorch Lightning version:", pytorch_lightning.__version__)

import rdkit
print("RDKit version:", rdkit.__version__)


/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/libpyg.so: undefined symbol: _ZNK5torch8autograd4Node4nameEv
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings

PyG version: 2.6.0
PyTorch Lightning version: 2.4.0
RDKit version: 2022.09.5


In [8]:
print("Torch version:", torch.__version__)

Torch version: 2.4.0


# Running their model- GPSConv

In [9]:
# %cd /kaggle/working/GraphGPS
# !WANDB_MODE=disabled python main.py --cfg configs/GPS/ogbg-molhiv-GPS.yaml

# Our Model - WeightedGraphGPS

In [17]:
%cd /kaggle/working
!rm -rf WeightedGraphGPS

/kaggle/working


In [18]:
#clone for our project
%cd /kaggle/working
!git clone --branch concat_two_layers https://github.com/omertalmi5/WeightedGraphGPS.git

/kaggle/working
Cloning into 'WeightedGraphGPS'...
remote: Enumerating objects: 617, done.
remote: Counting objects: 100% (409/409), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 617 (delta 324), reused 304 (delta 283), pack-reused 208 (from 1)
Receiving objects: 100% (617/617), 12.95 MiB | 35.65 MiB/s, done.
Resolving deltas: 100% (406/406), done.


In [19]:
!pip install wandb

In [20]:
!wandb login 5f1c292fde40243d281618fe88d712346cfe90d7

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [21]:
import os
# Set the environment variable
os.environ['USERNAME'] = "omertalmi-tel-aviv-university"

In [22]:
%cd /kaggle/working/WeightedGraphGPS
!WANDB_MODE=online python main.py --cfg configs/GPS/ogbg-molhiv-GPS.yaml wandb.use True wandb.entity $USERNAME

/kaggle/working/WeightedGraphGPS
[*] Run ID 0: seed=0, split_index=0
    Starting now: 2024-09-26 06:55:19.253844
Downloaded 0.00 GB: 100%|█████████████████████████| 3/3 [00:00<00:00,  4.07it/s]
Extracting ./datasets/hiv.zip
Processing...
Loading necessary files...
This might take a while.
Processing graphs...
100%|██████████████████████████████████| 41127/41127 [00:00<00:00, 66679.19it/s]
Converting graphs into PyG objects...
100%|██████████████████████████████████| 41127/41127 [00:02<00:00, 18862.44it/s]
Saving...
Done!
/opt/conda/lib/python3.10/site-packages/ogb/graphproppred/dataset_pyg.py:68: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weigh